<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Missing-Values" data-toc-modified-id="Missing-Values-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Missing Values</a></span></li><li><span><a href="#Exploratory-Analysis" data-toc-modified-id="Exploratory-Analysis-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Exploratory Analysis</a></span></li><li><span><a href="#Define-Churn" data-toc-modified-id="Define-Churn-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Define Churn</a></span></li><li><span><a href="#Feature-Engineering" data-toc-modified-id="Feature-Engineering-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Feature Engineering</a></span><ul class="toc-item"><li><span><a href="#downgrades-page" data-toc-modified-id="downgrades-page-4.1"><span class="toc-item-num">4.1&nbsp;&nbsp;</span>downgrades page</a></span></li><li><span><a href="#Add-friend" data-toc-modified-id="Add-friend-4.2"><span class="toc-item-num">4.2&nbsp;&nbsp;</span>Add friend</a></span></li><li><span><a href="#total-length-of-time-listened" data-toc-modified-id="total-length-of-time-listened-4.3"><span class="toc-item-num">4.3&nbsp;&nbsp;</span>total length of time listened</a></span></li><li><span><a href="#Number-of-songs-listened" data-toc-modified-id="Number-of-songs-listened-4.4"><span class="toc-item-num">4.4&nbsp;&nbsp;</span>Number of songs listened</a></span></li><li><span><a href="#Number-of-sessions-per-user" data-toc-modified-id="Number-of-sessions-per-user-4.5"><span class="toc-item-num">4.5&nbsp;&nbsp;</span>Number of sessions per user</a></span></li><li><span><a href="#Average-songs-per-seesion" data-toc-modified-id="Average-songs-per-seesion-4.6"><span class="toc-item-num">4.6&nbsp;&nbsp;</span>Average songs per seesion</a></span></li><li><span><a href="#average-session-per-day/month" data-toc-modified-id="average-session-per-day/month-4.7"><span class="toc-item-num">4.7&nbsp;&nbsp;</span>average session per day/month</a></span></li><li><span><a href="#location-feature" data-toc-modified-id="location-feature-4.8"><span class="toc-item-num">4.8&nbsp;&nbsp;</span>location feature</a></span></li><li><span><a href="#os-and-browser-features" data-toc-modified-id="os-and-browser-features-4.9"><span class="toc-item-num">4.9&nbsp;&nbsp;</span>os and browser features</a></span></li><li><span><a href="#label" data-toc-modified-id="label-4.10"><span class="toc-item-num">4.10&nbsp;&nbsp;</span>label</a></span></li></ul></li><li><span><a href="#Modeling" data-toc-modified-id="Modeling-5"><span class="toc-item-num">5&nbsp;&nbsp;</span>Modeling</a></span><ul class="toc-item"><li><span><a href="#Pipeline" data-toc-modified-id="Pipeline-5.1"><span class="toc-item-num">5.1&nbsp;&nbsp;</span>Pipeline</a></span></li></ul></li></ul></div>

In [2]:
%%configure -f
{ "conf":{
"spark.pyspark.python": "python3",
"spark.pyspark.virtualenv.enabled": "true",
"spark.pyspark.virtualenv.type":"native",
"spark.pyspark.virtualenv.bin.path":"/usr/bin/virtualenv"
}}

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
4,application_1567980939812_0005,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
4,application_1567980939812_0005,pyspark,idle,Link,Link,✔


In [3]:
sc.install_pypi_package("pandas")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

  Using cached https://files.pythonhosted.org/packages/73/9b/52e228545d14f14bb2a1622e225f38463c8726645165e1cb7dde95bfe6d4/pandas-0.25.1-cp36-cp36m-manylinux1_x86_64.whl
  Using cached https://files.pythonhosted.org/packages/41/17/c62faccbfbd163c7f57f3844689e3a78bae1f403648a6afb1d0866d87fbb/python_dateutil-2.8.0-py2.py3-none-any.whl

In [4]:
from pyspark.sql import SparkSession, Window
from pyspark.sql.functions import udf, col, count, avg, stddev, isnull, when, isnan 
from pyspark.sql.functions import desc, countDistinct, min, max
from pyspark.sql.functions import sum as Fsum
from pyspark.sql.types import IntegerType, DateType, StringType
from pyspark.ml.feature import StringIndexer, VectorAssembler, StandardScaler
from pyspark.ml import Pipeline
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.classification import LogisticRegression, RandomForestClassifier, GBTClassifier
from pyspark.ml.tuning import CrossValidator, ParamGridBuilder

from datetime import datetime
import re
import string
from time import time
import pandas as pd

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [10]:
# Create a Spark session
spark = SparkSession \
    .builder \
    .appName("sparkify") \
    .getOrCreate()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [12]:
# Read in full sparkify dataset
event_data = "s3n://udacity-dsnd/sparkify/sparkify_event_data.json"

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [13]:
customer_event_df = spark.read.json(event_data)
customer_event_df.head()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Row(artist='Popol Vuh', auth='Logged In', firstName='Shlok', gender='M', itemInSession=278, lastName='Johnson', length=524.32934, level='paid', location='Dallas-Fort Worth-Arlington, TX', method='PUT', page='NextSong', registration=1533734541000, sessionId=22683, song='Ich mache einen Spiegel - Dream Part 4', status=200, ts=1538352001000, userAgent='"Mozilla/5.0 (Windows NT 6.1; WOW64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/36.0.1985.143 Safari/537.36"', userId='1749042')

In [7]:
customer_event_df.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- artist: string (nullable = true)
 |-- auth: string (nullable = true)
 |-- firstName: string (nullable = true)
 |-- gender: string (nullable = true)
 |-- itemInSession: long (nullable = true)
 |-- lastName: string (nullable = true)
 |-- length: double (nullable = true)
 |-- level: string (nullable = true)
 |-- location: string (nullable = true)
 |-- method: string (nullable = true)
 |-- page: string (nullable = true)
 |-- registration: long (nullable = true)
 |-- sessionId: long (nullable = true)
 |-- song: string (nullable = true)
 |-- status: long (nullable = true)
 |-- ts: long (nullable = true)
 |-- userAgent: string (nullable = true)
 |-- userId: string (nullable = true)

In [8]:
print(f"The shape of the customer event susbset {(customer_event_df.count(), len(customer_event_df.columns))}")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

The shape of the customer event susbset (26259199, 18)

In [9]:
def counting_null(df):
    """
    printthe key value pairs of columns and their missing values.
    :param df: pyspark dataframe
    :return null_count: dict. the number of null values in each column
    """
    
    for col in df.columns:
        missing_values = df.filter((df[col] == "") \
                                   | df[col].isNull()
                                   | isnan(df[col])).count()
        print(f"{col}:{missing_values}")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## Missing Values

Column firstName, gender, lastNam, registration, location, userAgent and userId have the same amount of missing values. All of the missing values are occurred in the same places. So we can removed all these missing values.
We will not remove the null values in artist, length, song column as some people may just login the app and not listen to any song.

In [10]:
customer_event_df = customer_event_df.dropna(how="any", subset=["userId", "sessionId"])
customer_event_df = customer_event_df.filter(customer_event_df.userId != "")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [11]:
customer_event_df = customer_event_df.dropna(how="any", subset=["firstName"])

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [12]:
customer_event_df = customer_event_df.fillna('None', subset=['artist', 'song'])
customer_event_df = customer_event_df.fillna(0, subset=['length'])

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [13]:
#counting_null(customer_event_df)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## Exploratory Analysis

In [14]:
customer_event_df.select("artist", "userId").groupby("artist"). \
    agg({'artist': 'count'}).select('artist', 'count(artist)'). \
    sort(desc('count(artist)')).show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+-------------+
|              artist|count(artist)|
+--------------------+-------------+
|                None|      4630448|
|       Kings Of Leon|       169824|
|            Coldplay|       164369|
|Florence + The Ma...|       115348|
|       Dwight Yoakam|       107284|
|                Muse|       104710|
|            BjÃÂ¶rk|       104654|
|      The Black Keys|       102694|
|       Justin Bieber|       100219|
|        Jack Johnson|        94263|
|              Eminem|        84548|
|           Radiohead|        78407|
|               Train|        78109|
|     Alliance Ethnik|        77870|
|         OneRepublic|        77752|
|         The Killers|        75918|
|        Taylor Swift|        75400|
|         Linkin Park|        71639|
|         Evanescence|        71462|
|           Metallica|        66631|
+--------------------+-------------+
only showing top 20 rows

In [15]:
unique_count_artist = customer_event_df.select("artist").distinct().count()
print(f"There are a total of {unique_count_artist} artists in this dataset")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

There are a total of 38338 artists in this dataset

In [16]:
customer_event_df.select("auth", ).groupby("auth"). \
    agg({'auth': 'count'}).select('auth', col('count(auth)').alias('total')). \
    sort(desc('total')).show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---------+--------+
|     auth|   total|
+---------+--------+
|Logged In|25475717|
|Cancelled|    5003|
+---------+--------+

In [17]:
customer_event_df.select("itemInSession").describe().show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+------------------+
|summary|     itemInSession|
+-------+------------------+
|  count|          25480720|
|   mean|107.03340196038417|
| stddev|117.76563663964197|
|    min|                 0|
|    max|              1426|
+-------+------------------+

In [18]:
customer_event_df.select("method").groupby("method"). \
    agg({'method': 'count'}).select('method', col('count(method)').alias('total')). \
    sort(desc('total')).show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+------+--------+
|method|   total|
+------+--------+
|   PUT|23572687|
|   GET| 1908033|
+------+--------+

In [19]:
customer_event_df.select(countDistinct('userId').alias('number_of_userId'), 
                         countDistinct('sessionId').alias('number_of_sessionId')).show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+----------------+-------------------+
|number_of_userId|number_of_sessionId|
+----------------+-------------------+
|           22277|             223096|
+----------------+-------------------+

In [20]:
customer_event_df.select("song", 'artist').groupby("song"). \
    agg({'song': 'count'}).select('song', col('count(song)').alias('total')). \
    sort(desc('total')).show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+-------+
|                song|  total|
+--------------------+-------+
|                None|4630450|
|      You're The One| 108085|
|                Undo|  94183|
|             Revelry|  77588|
|       Sehr kosmisch|  62450|
|Horn Concerto No....|  57267|
|Dog Days Are Over...|  52160|
|             Secrets|  43483|
|        Use Somebody|  42670|
|              Canada|  40765|
|             Invalid|  39253|
|    Ain't Misbehavin|  35915|
|       ReprÃÂ©sente|  35557|
|Catch You Baby (S...|  34561|
|    Somebody To Love|  34228|
|SinceritÃÂ© Et J...|  33006|
|              Yellow|  32536|
|    Hey_ Soul Sister|  30334|
|           Fireflies|  29185|
|            The Gift|  28783|
+--------------------+-------+
only showing top 20 rows

In [21]:
customer_event_df.filter(col('song') == 'None').groupby('page'). \
    agg({'page': 'count'}).sort((desc('count(page)'))).show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------------------+-----------+
|                page|count(page)|
+--------------------+-----------+
|           Thumbs Up|    1151465|
|                Home|     933124|
|     Add to Playlist|     597921|
|         Roll Advert|     385212|
|          Add Friend|     381664|
|              Logout|     296005|
|         Thumbs Down|     239212|
|           Downgrade|     184240|
|            Settings|     147074|
|                Help|     129448|
|             Upgrade|      50507|
|               About|      48377|
|       Save Settings|      29516|
|               Error|      25048|
|      Submit Upgrade|      15135|
|    Submit Downgrade|       6494|
|Cancellation Conf...|       5003|
|              Cancel|       5003|
|            NextSong|          2|
+--------------------+-----------+

In [22]:
useragent_count = customer_event_df.select("userAgent").groupby("userAgent"). \
    agg({'userAgent': 'count'}).select('userAgent', col('count(userAgent)').alias('total')). \
    sort(desc('total')).toPandas()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [14]:
def os_feature(s):
    os = re.findall("Ubuntu|Windows|Mac|iPad|iPhone|Linux", str(s))
    if os:
        os = os[0].translate(str.maketrans('', '', string.punctuation)).strip()
    
    return os

def browser_feature(s):
    browser_list = re.findall("Chrome[/\w+.]+|Safari[/\w.]+|Mobile[/\w.]+|Firefox[/\w.]+|Trident[/\w.]+", str(s))
    if browser_list:
        browser_list = browser_list[0].strip().split("/")[0]
    
    return browser_list

def browser_version_feature(s):
    version_list = re.findall("Chrome[/\w+.]+|Safari[/\w.]+|Mobile[/\w.]+|Firefox[/\w.]+|Trident[/\w.]+", str(s))
    if version_list:
        version_list = version_list[0].strip().split("/")[1]
    
    return version_list

os_udf = udf(os_feature, StringType())
browser_udf = udf(browser_feature, StringType())
version_udf = udf(browser_version_feature, StringType())

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [15]:
customer_event_df = customer_event_df.withColumn("os",
                                                 os_udf(customer_event_df.userAgent))

customer_event_df = customer_event_df.withColumn("browser",
                                                 browser_udf(customer_event_df.userAgent))
customer_event_df = customer_event_df.withColumn("browser_ver",
                                                 version_udf(customer_event_df.userAgent))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [25]:
customer_event_df.select("browser").groupby("browser"). \
    agg({'browser': 'count'}).select('browser', col('count(browser)').alias('total')). \
    sort(desc('total')).show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+--------+
|browser|   total|
+-------+--------+
| Chrome|13617087|
|Firefox| 5865572|
| Safari| 3351489|
|Trident| 1342097|
| Mobile| 1304475|
+-------+--------+

In [26]:
customer_event_df.select("browser_ver").groupby("browser_ver"). \
    agg({'browser_ver': 'count'}).select('browser_ver', col('count(browser_ver)').alias('total')). \
    sort(desc('total')).show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------------+-------+
|  browser_ver|  total|
+-------------+-------+
|36.0.1985.143|6355492|
|         31.0|5410255|
|36.0.1985.125|4896580|
| 37.0.2062.94|1716235|
|     537.78.2|1418844|
|     537.77.4|1286229|
|       11D257| 977863|
|          7.0| 920112|
|35.0.1916.153| 256355|
|       11D201| 255883|
|         30.0| 236017|
|37.0.2062.103| 221846|
|          6.0| 215797|
|          5.0| 206188|
|    534.59.10| 154934|
|    537.75.14| 120164|
|      600.1.8| 119180|
|37.0.2062.102| 103929|
|       538.46|  98817|
|         32.0|  98586|
+-------------+-------+
only showing top 20 rows

In [27]:
customer_event_df.select("os").groupby("os"). \
    agg({'os': 'count'}).select('os', col('count(os)').alias('total')). \
    sort(desc('total')).show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+--------+
|     os|   total|
+-------+--------+
|Windows|12425943|
|    Mac|10101196|
|  Linux| 1103954|
| iPhone|  939025|
| Ubuntu|  545152|
|   iPad|  365450|
+-------+--------+

## Define Churn

* There are 5003 cancellation of subscription in 12GB dataset.
* We create a churn label column for the dataset. It returns 1 if Cancecllation Confirmation events happens.

In [28]:
customer_event_df.filter(col("page") == "Cancellation Confirmation" ). \
    select("userId").distinct().count()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

5003

In [16]:
is_cancellation = udf(lambda x: 1 if x == "Cancellation Confirmation" else 0, IntegerType())

customer_event_df = customer_event_df.withColumn("churn", is_cancellation("page"))
# there are users that cancelled their subscription in one of the session 
# we have to assign 1 to these users
window_func = Window.partitionBy("userId").rangeBetween(Window.unboundedPreceding,
                                                        Window.unboundedFollowing)
customer_event_df = customer_event_df.withColumn("churn", Fsum("churn").over(window_func))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [30]:
# ts feature
customer_event_df.select("ts").describe().show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+--------------------+
|summary|                  ts|
+-------+--------------------+
|  count|            25480720|
|   mean|1.540907380054355...|
| stddev|1.5154756164254992E9|
|    min|       1538352001000|
|    max|       1543622402000|
+-------+--------------------+

In [31]:
start_ts = int(1538352011000) / 1000
start_date = datetime.utcfromtimestamp(start_ts).strftime("%Y-%m-%d %H:%M:%S")
print(f"Start Time: {start_date}")

end_ts = int(1543622466000) / 1000
end_date = datetime.utcfromtimestamp(end_ts).strftime("%Y-%m-%d %H:%M:%S")
print(f"End Time: {end_date}")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

Start Time: 2018-10-01 00:00:11
End Time: 2018-12-01 00:01:06

In [17]:
ts_to_day_fn = udf(lambda ts: int(datetime.fromtimestamp(ts / 1000.0).strftime("%w")), IntegerType())
ts_to_hour_fn = udf(lambda ts: datetime.fromtimestamp(ts / 1000.0).hour, IntegerType())
ts_to_date_fn = udf(lambda ts: datetime.fromtimestamp(ts / 1000.0).date(), DateType())
ts_to_month_fn = udf(lambda ts: datetime.fromtimestamp(ts / 1000.0).month, IntegerType())

customer_event_df = customer_event_df.withColumn("event_weekday", 
                                                 ts_to_day_fn(customer_event_df.ts))
customer_event_df = customer_event_df.withColumn("event_hour", 
                                                 ts_to_hour_fn(customer_event_df.ts))
customer_event_df = customer_event_df.withColumn("event_month",
                                                 ts_to_month_fn(customer_event_df.ts))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [18]:
customer_event_df = customer_event_df.withColumn("event_date",
                                                 ts_to_date_fn(customer_event_df.ts))
customer_event_df = customer_event_df.withColumn("registration_date",
                                                 ts_to_date_fn(customer_event_df.registration))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [19]:
def clean_location(s):
    
    location = s.split(",")[1].strip()
    
    if len(location) >= 3:
        location = location.split("-")[0].strip()
    return location

clean_location_udf = udf(clean_location, StringType())

customer_event_df = customer_event_df.withColumn("location",
                                                 clean_location_udf(customer_event_df.location))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [35]:
# create a temp view from customer_even_df to use SQL type of syntax for data analysis
customer_event_df.createOrReplaceTempView('event');

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [36]:
churn_count_df = spark.sql("""
                           SELECT churn, 
                                  Count(DISTINCT userid) AS total_user 
                           FROM   event 
                           GROUP  BY churn 
                           """).toPandas()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [37]:
churn_count_df["percentage"] = churn_count_df["total_user"] / churn_count_df["total_user"].sum() * 100
churn_count_df

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

   churn  total_user  percentage
0      0       17274   77.541859
1      1        5003   22.458141

In [38]:
gender_count = customer_event_df.select("gender").groupby("gender"). \
    agg({'gender': 'count'}).select('gender', col('count(gender)').alias('total')). \
    sort(desc('total')).toPandas()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [39]:
gender_count["percentage"] = gender_count["total"] / gender_count["total"].sum() * 100
gender_count

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

  gender     total  percentage
0      M  13299562   52.194608
1      F  12181158   47.805392

In [40]:
level_count = customer_event_df.select("level").groupby("level"). \
    agg({'level': 'count'}).select('level', col('count(level)').alias('total')). \
    sort(desc('total')).toPandas()
level_count["percentage"] = level_count["total"] / level_count["total"].sum() * 100
level_count

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

  level     total  percentage
0  paid  20072480   78.775168
1  free   5408240   21.224832

In [41]:
location_df = spark.sql("""
                           SELECT location, 
                                  Count(DISTINCT userid) AS total_user 
                           FROM   event 
                           GROUP  BY location 
                           ORDER BY total_user
                           DESC
                           """).toPandas()

location_churn_df = spark.sql("""
                           SELECT location, 
                                  Count(DISTINCT userid) AS churn_user 
                           FROM   event 
                           WHERE churn=1
                           GROUP BY location
                           """).toPandas()

merge_location_df = location_churn_df.merge(location_df, on='location').sort_values(by='total_user', ascending=False)
merge_location_df['percentage'] = merge_location_df['churn_user'] / merge_location_df['total_user'] * 100

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [42]:
merge_location_df.sort_values(by='percentage', ascending=False)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

   location  churn_user  total_user  percentage
37       SD          17          52   32.692308
26       ME          19          65   29.230769
10       WY           9          32   28.125000
40       WV          34         126   26.984127
11       NH          18          67   26.865672
50       HI          31         116   26.724138
5        DC         119         455   26.153846
24       MO         126         497   25.352113
15       ID          29         115   25.217391
14       WI          84         335   25.074627
20       NC         159         638   24.921630
31       IN          96         390   24.615385
47       AR          48         197   24.365482
42       MA         124         510   24.313725
21       VT           8          33   24.242424
8        RI          32         133   24.060150
48       OK          54         227   23.788546
46       AK           8          34   23.529412
23       DE           3          13   23.076923
41       GA         159         690   23

In [43]:
page_count = customer_event_df.select("page").groupby("page"). \
    agg({'page': 'count'}).select('page', col('count(page)').alias('total')). \
    sort(desc('total')).toPandas()

page_count['percentage'] = page_count['total']/ page_count['total'].sum()

page_count

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

                         page     total  percentage
0                    NextSong  20850272    0.818276
1                   Thumbs Up   1151465    0.045190
2                        Home    933124    0.036621
3             Add to Playlist    597921    0.023466
4                 Roll Advert    385212    0.015118
5                  Add Friend    381664    0.014979
6                      Logout    296005    0.011617
7                 Thumbs Down    239212    0.009388
8                   Downgrade    184240    0.007231
9                    Settings    147074    0.005772
10                       Help    129448    0.005080
11                    Upgrade     50507    0.001982
12                      About     48377    0.001899
13              Save Settings     29516    0.001158
14                      Error     25048    0.000983
15             Submit Upgrade     15135    0.000594
16           Submit Downgrade      6494    0.000255
17  Cancellation Confirmation      5003    0.000196
18          

In [44]:
day_user_churn_df = spark.sql("""
                    SELECT event_weekday,
                           churn,
                           COUNT(DISTINCT(userId)) AS total_user
                    FROM   event
                    GROUP BY event_weekday,
                             churn
                    """).toPandas()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [45]:
day_user_churn_df.sort_values(by="event_weekday")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

    event_weekday  churn  total_user
3               0      0       10441
4               0      1        2762
0               1      0       13481
7               1      1        3973
8               2      1        3925
11              2      0       13840
2               3      1        3807
10              3      0       13822
5               4      0       13595
6               4      1        3737
1               5      1        3589
13              5      0       13744
9               6      0       10906
12              6      1        2872

## Feature Engineering

In [20]:
count_artist = customer_event_df \
    .filter(col("page") == "NextSong") \
    .select("userId", "artist") \
    .dropDuplicates() \
    .groupby("userId") \
    .count() \
    .withColumnRenamed('count', 'total_artist') \

count_artist.show(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+------------+
| userId|total_artist|
+-------+------------+
|1500901|        2252|
|1338783|        1162|
|1178731|        1684|
|1331962|         504|
|1333041|         577|
+-------+------------+
only showing top 5 rows

In [21]:
# Binarize the gender
binarize_gender = customer_event_df \
    .select("userId", "gender") \
    .dropDuplicates() \
    .replace(["M", "F"], ["0", "1"], "gender") \
    .select("userId", col("gender").cast("int"))

binarize_gender.show(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+------+
| userId|gender|
+-------+------+
|1005442|     1|
|1163054|     0|
|1180653|     0|
|1368451|     0|
|1859039|     0|
+-------+------+
only showing top 5 rows

In [22]:
#Binarize level
binarize_level = customer_event_df \
    .select("userId", "level") \
    .dropDuplicates() \
    .replace(["free", "paid"], ["0", "1"], "level") \
    .select("userId", col("level").cast("int"))

binarize_level.show(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+-----+
| userId|level|
+-------+-----+
|1108378|    0|
|1140592|    0|
|1057440|    1|
|1029700|    1|
|1808914|    0|
+-------+-----+
only showing top 5 rows



### downgrades page

In [23]:
downgrade_feat = customer_event_df \
    .select("userId", "page") \
    .where(col("page") == "Downgrade") \
    .groupby("userId") \
    .count() \
    .withColumnRenamed("count", "num_downgrade")

downgrade_feat.show(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+-------------+
| userId|num_downgrade|
+-------+-------------+
|1390009|            4|
|1500901|           41|
|1178731|           23|
|1083324|           17|
|1965481|           16|
+-------+-------------+
only showing top 5 rows

In [24]:
playlist_feat = customer_event_df \
    .select("userId", "page") \
    .where(col("page") == "Add to Playlist") \
    .groupby("userId") \
    .count() \
    .withColumnRenamed("count", "num_song_playlist")

playlist_feat.show(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+-----------------+
| userId|num_song_playlist|
+-------+-----------------+
|1588738|               57|
|1396828|               50|
|1069552|               11|
|1178731|               84|
|1142513|               19|
+-------+-----------------+
only showing top 5 rows

In [25]:
thumbs_down_feat = customer_event_df \
    .select("userId", "page") \
    .where(col("page") == "Thumbs Down") \
    .groupby("userId") \
    .count() \
    .withColumnRenamed("count", "num_thumbs_down")

thumbs_down_feat.show(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+---------------+
| userId|num_thumbs_down|
+-------+---------------+
|1069552|              6|
|1875484|             14|
|1588738|             22|
|1396828|             10|
|1178731|             27|
+-------+---------------+
only showing top 5 rows

In [26]:
thumbs_up_feat = customer_event_df \
    .select("userId", "page") \
    .where(col("page") == "Thumbs Up") \
    .groupby("userId") \
    .count() \
    .withColumnRenamed("count", "num_thumbs_up")

thumbs_up_feat.show(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+-------------+
| userId|num_thumbs_up|
+-------+-------------+
|1069552|           26|
|1658815|           53|
|1178731|          147|
|1718034|           30|
|1875484|           81|
+-------+-------------+
only showing top 5 rows

### Add friend

In [27]:
friend_feat = customer_event_df \
    .select("userId", "page") \
    .where(col("page") == "Add Friend") \
    .groupby("userId") \
    .count() \
    .withColumnRenamed("count", "num_friend")

friend_feat.show(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+----------+
| userId|num_friend|
+-------+----------+
|1983423|        23|
|1002185|        25|
|1190352|        40|
|1142513|        16|
|1083324|        26|
+-------+----------+
only showing top 5 rows

### total length of time listened

In [28]:
total_length = customer_event_df \
    .select("userId", "length") \
    .groupby("userId") \
    .sum() \
    .withColumnRenamed("sum(length)", "total_length")

total_length.show(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+------------------+
| userId|      total_length|
+-------+------------------+
|1390009| 44514.53022999999|
|1519090| 78136.59243000002|
|1394508|308561.98851999996|
|1178731| 660566.6424100001|
|1351489|260582.08369999996|
+-------+------------------+
only showing top 5 rows

### Number of songs listened

In [29]:
total_songs = customer_event_df \
    .select("userId", "song") \
    .groupby("userId") \
    .count() \
    .withColumnRenamed("count", "num_song")

total_songs.show(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+--------+
| userId|num_song|
+-------+--------+
|1331962|     727|
|1178731|    3165|
|1528396|     154|
|1002185|    2080|
|1333041|     966|
+-------+--------+
only showing top 5 rows

### Number of sessions per user

In [30]:
num_session = customer_event_df \
    .select("userId", "sessionId") \
    .dropDuplicates() \
    .groupby("userId") \
    .count() \
    .withColumnRenamed("count", "session")

num_session.show(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+-------+
| userId|session|
+-------+-------+
|1338783|     16|
|1271218|     51|
|1492713|     17|
|1875484|     25|
|1770964|     16|
+-------+-------+
only showing top 5 rows

### Average songs per seesion

In [31]:
avg_songs_session = customer_event_df.filter(col("page") == "NextSong") \
    .groupby("userId", "sessionId") \
    .count() \
    .groupby("userId") \
    .agg({"count": "avg"}) \
    .withColumnRenamed("avg(count)", "avg_songs")

avg_songs_session.show(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+-----------------+
| userId|        avg_songs|
+-------+-----------------+
|1875484|            59.96|
|1396135|           121.75|
|1057724|98.64102564102564|
|1624220|          27.6875|
|1828442|21.09090909090909|
+-------+-----------------+
only showing top 5 rows

### average session per day/month

In [32]:
daily_sess = customer_event_df.groupby(["userId", "event_weekday"]) \
    .agg(countDistinct("sessionId")) \
    .groupby("userId").avg("count(DISTINCT sessionId)") \
    .withColumnRenamed("avg(count(DISTINCT sessionId))", "avg_daily_sessions")
daily_sess.show(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+------------------+
| userId|avg_daily_sessions|
+-------+------------------+
|1853227| 2.857142857142857|
|1676292| 2.142857142857143|
|1500901|               6.0|
|1200956| 7.285714285714286|
|1567400|               1.5|
+-------+------------------+
only showing top 5 rows

In [33]:
month_sess = customer_event_df.groupby(["userId", "event_month"]) \
    .agg(countDistinct("sessionId")) \
    .groupby("userId").avg("count(DISTINCT sessionId)") \
    .withColumnRenamed("avg(count(DISTINCT sessionId))", "avg_monthly_sessions")
month_sess.show(5)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+--------------------+
| userId|avg_monthly_sessions|
+-------+--------------------+
|1396135|                 4.5|
|1663036|                11.5|
|1965481|                15.5|
|1853227|                 9.0|
|1617595|                 1.0|
+-------+--------------------+
only showing top 5 rows

### location feature

In [34]:
location_feat = customer_event_df \
    .select("userId", "location") \
    .dropDuplicates()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

### os and browser features

In [35]:
os_feat = customer_event_df \
    .select("userId", "os") \
    .dropDuplicates()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [36]:
browser_feat = customer_event_df \
    .select("userId", "browser") \
    .dropDuplicates()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [37]:
browser_ver_feat = customer_event_df \
    .select("userId", "browser_ver") \
    .dropDuplicates()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

### label

In [38]:
label = customer_event_df \
    .select("userId", col("churn").alias("label")) \
    .dropDuplicates() \
    .orderBy("userId")

label.describe().show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+------------------+-------------------+
|summary|            userId|              label|
+-------+------------------+-------------------+
|  count|             22278|              22278|
|   mean|1498782.9615764432|0.22457132597181076|
| stddev| 288851.8472659187|0.41730907312356175|
|    min|           1000025|                  0|
|    max|           1999996|                  1|
+-------+------------------+-------------------+

In [39]:
transformed_event = count_artist.join(binarize_gender, "userId", "outer") \
    .join(binarize_level, "userId", "outer") \
    .join(downgrade_feat, "userId", "outer") \
    .join(playlist_feat, "userId", "outer") \
    .join(thumbs_down_feat, "userId", "outer") \
    .join(thumbs_up_feat, "userId", "outer") \
    .join(friend_feat, "userId", "outer") \
    .join(total_length, "userId", "outer") \
    .join(num_session, "userId", "outer") \
    .join(avg_songs_session, "userId", "outer") \
    .join(daily_sess, "userId", "outer") \
    .join(month_sess, "userId", "outer") \
    .join(os_feat, "userId", "outer") \
    .join(browser_feat, "userId", "outer") \
    .join(browser_ver_feat, "userId", "outer") \
    .join(label, "userId", "outer") \
    .drop("userId") \
    .fillna(0)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## Modeling

In [40]:
transformed_event.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- total_artist: long (nullable = true)
 |-- gender: integer (nullable = true)
 |-- level: integer (nullable = true)
 |-- num_downgrade: long (nullable = true)
 |-- num_song_playlist: long (nullable = true)
 |-- num_thumbs_down: long (nullable = true)
 |-- num_thumbs_up: long (nullable = true)
 |-- num_friend: long (nullable = true)
 |-- total_length: double (nullable = false)
 |-- session: long (nullable = true)
 |-- avg_songs: double (nullable = false)
 |-- avg_daily_sessions: double (nullable = false)
 |-- avg_monthly_sessions: double (nullable = false)
 |-- os: string (nullable = true)
 |-- browser: string (nullable = true)
 |-- browser_ver: string (nullable = true)
 |-- label: long (nullable = true)

### Pipeline

In [41]:
num_cols= ["total_artist", "gender", "level", "num_downgrade", "num_song_playlist",
           "num_thumbs_down", "num_thumbs_up", "num_friend", "total_length",
           "session", "avg_songs", "avg_daily_sessions", "avg_monthly_sessions"]


indexer_os = StringIndexer(inputCol="os", outputCol="os_index")
indexer_browser = StringIndexer(inputCol="browser", outputCol="browser_index")
indexer_browser_ver = StringIndexer(inputCol="browser_ver", outputCol="browser_ver_index")
assemblerCat = VectorAssembler(inputCols=["os_index", "browser_index", "browser_ver_index"], outputCol="cat")

pipelineCat = Pipeline(stages = [indexer_os, indexer_browser, indexer_browser_ver, assemblerCat])
transformed_event = pipelineCat.fit(transformed_event).transform(transformed_event)



assemblerNum = VectorAssembler(inputCols=num_cols, outputCol="Num")
scaler = StandardScaler(inputCol="Num", outputCol="scaled_num")
pipelineNum = Pipeline(stages=[assemblerNum, scaler])
transformed_event = pipelineNum.fit(transformed_event).transform(transformed_event)

assembler = VectorAssembler(inputCols=["cat", "scaled_num"], outputCol="features")

features_pipeline = Pipeline(stages=[assembler])

transformed_event = features_pipeline.fit(transformed_event).transform(transformed_event)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [42]:
final_data = transformed_event.select(["label", "features"])


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [43]:
train_data, test_data = final_data.randomSplit([0.8, 0.2], seed=12)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [44]:
def feat_importance_df(model, cols, clf_name):
    """
    Create feature importance dataframe 
    :param model:  machine learning model
    :param cols: a list of columns 
    :param clf_name: str. the name of the classifier
    :return: feature_imp_df: dataframe of feature importance score 
    """
    feature_index = model.featureImportances.indices.tolist()
    feature_name = [cols[index] for index in feature_index]
    feature_imp = model.featureImportances.values.tolist()
    feature_imp_df = pd.DataFrame(list(zip(feature_name, feature_imp)), columns=["features", "importance_score"])
    feature_imp_df = feature_imp_df.sort_values("importance_score", ascending=False)

    return feature_imp_df

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [45]:
def evaluate_model(model, data):
    """
    Create summary metrics of machine learning model prediction
    :param model: Pyspark ml model 
    :param data: either test data or train data 
    :return: summary: accuracy, precision, recall and f1 score of the model
    """
    prediction = model.transform(data)
    
    evaluator = MulticlassClassificationEvaluator()
    
    eval_dict = {}
    eval_dict["accuracy"] = evaluator.evaluate(prediction, {evaluator.metricName: "accuracy"})
    eval_dict["precision"] = evaluator.evaluate(prediction, {evaluator.metricName: "weightedPrecision"})
    eval_dict["recall"] = evaluator.evaluate(prediction, {evaluator.metricName: "weightedRecall"})
    eval_dict["f1"] = evaluator.evaluate(prediction, {evaluator.metricName: "f1"})
    
    
    metrics = {k: round(v, 5) for k, v in eval_dict.items()}
    # create a spark datafrmae 
    summary = spark.createDataFrame(pd.DataFrame([metrics], columns=metrics.keys()))
    
    return summary

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [46]:
rf_model = RandomForestClassifier(labelCol="label", featuresCol="features")
param_grid_rf = ParamGridBuilder() \
    .addGrid(rf_model.maxDepth, [10]) \
    .build()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [47]:
start = time()
cross_val = CrossValidator(estimator=rf_model,
                           estimatorParamMaps=param_grid_rf,
                           evaluator=MulticlassClassificationEvaluator(),
                           numFolds=3)
clf = cross_val.fit(train_data)
summary = evaluate_model(clf, test_data)
summary.show()
end = time()
print(f"Training time approximately {(end - start) // 60} mins")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+--------+---------+-------+------+
|accuracy|precision| recall|    f1|
+--------+---------+-------+------+
| 0.84919|  0.85287|0.84919|0.8272|
+--------+---------+-------+------+

Training time approximately 312.0 mins